In [ ]:
%pip install transformers==4.28.1
%pip install datasets
%pip install sentencepiece
%pip install rouge_score
%pip install tabulate

In [2]:
import numpy as np
import pandas as pd
import re
import os
import sys
import json 
import ast
import pickle

sys.path.append('../../Quantlet/Create_description/')

import importlib
import preprocessing_utils
importlib.reload(preprocessing_utils)

import torch
import numpy as np
import datasets

from transformers import (
    AutoModelForSeq2SeqLM,
    AutoTokenizer,
    Seq2SeqTrainingArguments,
    Seq2SeqTrainer,
    DataCollatorForSeq2Seq
)

import nltk
from datetime import datetime

import evaluate
nltk.download("punkt", quiet=True)


from datasets import Dataset
from datasets import load_dataset


import evaluate
metric = evaluate.load("rouge")

from sklearn.model_selection import train_test_split, KFold

os.environ["WANDB_DISABLED"] = "true"
os.environ["CUDA_VISIBLE_DEVICES"]="0"


RS = 42

In [3]:
# LOAD created descriptions and the parsed Quantlets
parsed_Qs_file = '../../data/preprocessed/Quantlet/Parsed_Qs_19032023.pkl'
dataset = preprocessing_utils.QuantletDataset(parsed_Qs_file)

with open('../../data/preprocessed/Quantlet/Descriptions_Qs_19032023.pkl', 'rb') as f:
    descriptions = pickle.load(f)

# Identify the most common language in each folder containing metainfo file
dataset = dataset.parsed_Qs_file
dataset.type_script = dataset.type_script.str.replace('ipynb', 'py')
dataset['most_commen_lang'] = dataset.folder_name.map(dataset.groupby(['folder_name'])['type_script'].agg(pd.Series.mode))
dataset['most_commen_lang'] = dataset['most_commen_lang'].astype(str)

# create the additional index to merge discription back to the dataset
dataset['desc_idx'] = dataset.index

In [4]:
unique_metainfo_files = dataset.metainfo_file.astype(str).unique()

In [5]:
tocheck_data = dataset.copy()
tocheck_data.metainfo_file = tocheck_data.metainfo_file.astype(str)
tocheck_data['empty'] = (tocheck_data.metainfo_file=='empty').astype(int)
tocheck_data['empty_ratio'] = tocheck_data.folder_name.map(tocheck_data.groupby('folder_name').empty.sum()) / tocheck_data.folder_name.map(tocheck_data.groupby('folder_name').empty.count())

# identify the repos that do not have metainfo files at all
q_no_meta = tocheck_data[tocheck_data.empty_ratio==1].folder_name.unique()

In [6]:
# remove duplicates
Q_lang = dataset[['folder_name', 'most_commen_lang']].drop_duplicates().reset_index(drop=True)
Q_lang = Q_lang[~Q_lang.folder_name.isin(q_no_meta)]

In [7]:
# create the train and test set
labelled, test = train_test_split(Q_lang, test_size=0.2, random_state=RS, stratify=Q_lang.most_commen_lang)
print(labelled.shape, test.shape)

(2376, 2) (595, 2)


In [8]:
def get_metainfo_field(row, field='keywords'):
    try :
        row = ast.literal_eval(row)
    except:
        return 'empty'
    row = {k.lower():v for (k,v) in row.items()}
    if field in row.keys():
        return row[field]
    else:
        return 'empty'

In [9]:
# extract descriptions from metainfo file
dataset['meta_description'] = dataset.metainfo_file.apply(get_metainfo_field, field='description')

# remove with empty descriptions
dataset = dataset[~dataset.meta_description.isna()]

In [10]:
no_meta_ds = dataset.query('folder_name in @q_no_meta')
labelled_ds = dataset.query('folder_name in @labelled.folder_name')
test_ds = dataset.query('folder_name in @test.folder_name')

print(labelled_ds.shape, test_ds.shape)
full_shape = labelled_ds.shape[0] + test_ds.shape[0]
print(labelled_ds.shape[0] / full_shape, test_ds.shape[0]/full_shape)


no_meta_ds = no_meta_ds

(3865, 8) (968, 8)
0.7997103248499896 0.20028967515001034


In [13]:
labelled_ds = labelled_ds[labelled_ds.meta_description!='empty']
test_ds = test_ds[test_ds.meta_description!='empty']

#labelled_ds.description = labelled_ds.description.apply(lambda x : '; '.join(x))
#test_ds.description = test_ds.description.apply(lambda x : '; '.join(x))

In [14]:
# extract descriptions for train, test, no meta datasets
descriptions_labelled = [descriptions[i] for i in labelled_ds.desc_idx.values]
descriptions_test = [descriptions[i] for i in test_ds.desc_idx.values]
descriptions_no_meta = [descriptions[i] for i in no_meta_ds.desc_idx.values]

In [15]:
# Extract description lists
descriptions_labelled = [list(descr_dict.values()) for descr_dict in descriptions_labelled]
descriptions_test = [list(descr_dict.values()) for descr_dict in descriptions_test]
descriptions_no_meta = [list(descr_dict.values()) for descr_dict in descriptions_no_meta]

descriptions_labelled_list = [' \n '.join(descr[0]) for descr in descriptions_labelled]
descriptions_test_list = [' \n '.join(descr[0]) for descr in descriptions_test]

In [16]:
train_dataset_json = {'version' : '0.1.0',
                     'data' : [{'description': descriptions_labelled_list[i], 
                                'keywords' : labelled_ds.iloc[i]['meta_description']} for i in range(len(descriptions_labelled_list))]}

test_dataset_json = {'version' : '0.1.0',
                     'data' : [{'description': descriptions_test_list[i], 
                                'keywords' : test_ds.iloc[i]['meta_description']} for i in range(len(descriptions_test_list))]}


with open('../../data/preprocessed/Quantlet/labelled_dataset_descr.json', 'w') as f:
    json.dump(train_dataset_json, f)

with open('../../data/preprocessed/Quantlet/test_dataset_descr.json', 'w') as f:
    json.dump(test_dataset_json, f)

In [23]:
train_dataset_json['data'][0]['keywords']

'Applies the adaptive algorithm on the model proposed by Barrow (2006) to capture the disaster of stock market and forecast stock price.'

In [19]:
train_dataset_json['keywords']

KeyError: 'keywords'

In [17]:
train_dataset = load_dataset("json", data_files="../../data/preprocessed/Quantlet/labelled_dataset_descr.json", field="data")
test_dataset = load_dataset("json", data_files="../../data/preprocessed/Quantlet/test_dataset_descr.json", field="data")

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

DatasetGenerationError: An error occurred while generating the dataset

In [38]:
model_name = "sshleifer/distilbart-xsum-12-3"

model = AutoModelForSeq2SeqLM.from_pretrained(model_name)
tokenizer = AutoTokenizer.from_pretrained(model_name)

# tokenization
encoder_max_length = 512  # demo
decoder_max_length = 26

train_data_txt, validation_data_txt = train_dataset['train'].train_test_split(test_size=0.1).values()

In [39]:
a = [len(desc) for desc in train_data_txt['description']]
empty_descr_idx = np.where(np.array(a) == 0)


b = [len(desc) for desc in validation_data_txt['description']]
empty_descr_val_idx = np.where(np.array(b) == 0)

In [40]:
# create new dataset exluding those idx
train_data_txt = train_data_txt.select(
    (
        i for i in range(len(train_data_txt)) 
        if i not in set(empty_descr_idx[0])
    )
)

# create new dataset exluding those idx
validation_data_txt = validation_data_txt.select(
    (
        i for i in range(len(validation_data_txt)) 
        if i not in set(empty_descr_val_idx[0])
    )
)

Parameter 'indices'=<generator object <genexpr> at 0x7f218047fc10> of the transform datasets.arrow_dataset.Dataset.select couldn't be hashed properly, a random hash was used instead. Make sure your transforms and parameters are serializable with pickle or dill for the dataset fingerprinting and caching to work. If you reuse this transform, the caching mechanism will consider it to be different from the previous calls and recompute everything. This warning is only showed once. Subsequent hashing failures won't be showed.


In [41]:
def batch_tokenize_preprocess(batch, tokenizer, max_source_length, max_target_length):
    source, target = batch["description"], batch["keywords"]
    source_tokenized = tokenizer(
        source, padding="max_length", truncation=True, max_length=max_source_length
    )
    target_tokenized = tokenizer(
        target, padding="max_length", truncation=True, max_length=max_target_length
    )

    batch = {k: v for k, v in source_tokenized.items()}
    # Ignore padding in the loss
    batch["labels"] = [
        [-100 if token == tokenizer.pad_token_id else token for token in l]
        for l in target_tokenized["input_ids"]
    ]
    return batch


train_data = train_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=train_data_txt.column_names,
)

validation_data = validation_data_txt.map(
    lambda batch: batch_tokenize_preprocess(
        batch, tokenizer, encoder_max_length, decoder_max_length
    ),
    batched=True,
    remove_columns=validation_data_txt.column_names,
)

Map:   0%|          | 0/3447 [00:00<?, ? examples/s]

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [ ]:
def postprocess_text(preds, labels):

    preds = [pred.strip() for pred in preds]
    labels = [label.strip() for label in labels]

    # rougeLSum expects newline after each sentence
    preds = ["\n".join(nltk.sent_tokenize(pred)) for pred in preds]
    labels = ["\n".join(nltk.sent_tokenize(label)) for label in labels]

    return preds, labels

In [ ]:

def compute_metrics(eval_preds):
    
    preds, labels = eval_preds

    if isinstance(preds, tuple):
        preds = preds[0]

    decoded_preds = tokenizer.batch_decode(preds, skip_special_tokens=True)
    
    # Replace -100 in the labels as we can't decode them.
    labels = np.where(labels != -100, labels, tokenizer.pad_token_id)
    decoded_labels = tokenizer.batch_decode(labels, skip_special_tokens=True)

    # Some simple post-processing
    decoded_preds, decoded_labels = postprocess_text(decoded_preds, decoded_labels)
    
    result = metric.compute(
        predictions=decoded_preds, references=decoded_labels, use_stemmer=True
    )

    # Extract a few results from ROUGE
    result = {key: value * 100 for key, value in result.items()}

    prediction_lens = [
        np.count_nonzero(pred != tokenizer.pad_token_id) for pred in preds
    ]
    result["gen_len"] = np.mean(prediction_lens)
    result = {k: round(v, 4) for k, v in result.items()}
    return result

In [ ]:
training_args = Seq2SeqTrainingArguments(
    output_dir="results",
    num_train_epochs=5,  # demo
    do_train=True,
    do_eval=True,
    per_device_train_batch_size=16,  # demo
    per_device_eval_batch_size=16,
    # learning_rate=3e-05,
    warmup_steps=500,
    weight_decay=0.1,
    label_smoothing_factor=0.1,
    predict_with_generate=True,
    logging_dir="logs",
    logging_steps=100,
    save_total_limit=3,
    report_to=None
)

data_collator = DataCollatorForSeq2Seq(tokenizer, model=model)

trainer = Seq2SeqTrainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data,
    eval_dataset=validation_data,
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

Using the `WANDB_DISABLED` environment variable is deprecated and will be removed in v5. Use the --report_to flag to control the integrations used for logging result (for instance --report_to none).


In [ ]:
trainer.evaluate()

You're using a BartTokenizerFast tokenizer. Please note that with a fast tokenizer, using the `__call__` method is faster than using a method to encode the text followed by a call to the `pad` method to get a padded encoding.


{'eval_loss': 9.692809104919434,
 'eval_rouge1': 0.5463,
 'eval_rouge2': 0.035,
 'eval_rougeL': 0.5079,
 'eval_rougeLsum': 0.5026,
 'eval_gen_len': 17.8988,
 'eval_runtime': 23.0261,
 'eval_samples_per_second': 14.592,
 'eval_steps_per_second': 0.912}

In [42]:
trainer.train()

NameError: name 'trainer' is not defined

In [47]:
def generate_summary(test_samples, model):
    inputs = tokenizer(
        test_samples["description"],
        padding="max_length",
        truncation=True,
        max_length=encoder_max_length,
        return_tensors="pt",
    )
    input_ids = inputs.input_ids.to(model.device)
    attention_mask = inputs.attention_mask.to(model.device)
    outputs = model.generate(input_ids, attention_mask=attention_mask)
    output_str = tokenizer.batch_decode(outputs, skip_special_tokens=True)
    return outputs, output_str


model_before_tuning = AutoModelForSeq2SeqLM.from_pretrained(model_name)

test_samples = validation_data_txt.select(range(20))

summaries_before_tuning = generate_summary(test_samples, model_before_tuning)[1]
summaries_after_tuning = generate_summary(test_samples, model)[1]

In [48]:
print(
    
        list(zip(
            range(len(summaries_after_tuning)),
            summaries_after_tuning,
            summaries_before_tuning,
        ))
    
)
print("\nTarget summaries:\n")
print(
    list(enumerate(test_samples["keywords"])))


[(0, 'clustering; lasso; sentiment analysis', ' Here is a guide to some of the key questions used to ask.'), (1, 'asset; black-scholes; call; european-option; financial; option; option-price', ' A look back at some of the most memorable moments in the history of computing.'), (2, 'eigenvalues; principal component analysis; principal-components; visualization', ' Here is a guide to some of the keys.'), (3, 'asset; financial; gaussian; normal; visualization', ' A look at some of the key topics in the Spanish language.'), (4, 'normal-distribution; random; simulation; visualization', ' Here is a look at some of the key technologies used by the European Union.'), (5, 'asset; descriptive-statistics; distance; financial; portfolio; tree; visualization', ' A look at some of the key numbers on the subject of mathematics.'), (6, 'machine learning; model; prediction; simulation; time series', ' The YOEA112 project is the best solution to the firefly algorithm in MATLAB.'), (7, 'financial; log-ret